# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. 

Практическая работа разделена на 2 части: 
1. Построение нграмой языковой модели - основная часть, 12 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 4 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

In [1]:
import numpy as np
from collections import defaultdict
from typing import List, Dict, Tuple

# 1. Построение нграмной языковой модели. (12 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: 
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$

Поскольку униграмы не содержат в себе какого-дибо контекста, вероятность униграмы можно посчитать поделив кол-во этой слова на общее количество слов в обучающей выборке. 


In [2]:
# в первую очередь нам понадобится подсчитать статистику по обучающей выборке 
def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
    ngrams = defaultdict(int)
    # TODO реализуйте функцию, которая подсчитывает все 1-gram 2-gram ... order-gram ngram'ы в 
    for text in train_text:
        tokens = text.split()
        if bos:
            tokens = ['<s>'] + tokens
        if eos:
            tokens = tokens + ['</s>']

        for i in range(order):
            n_gram = i + 1
            for j in range(len(tokens)-n_gram+1):
                ngrams[tuple(tokens[j:j+n_gram])]+=1
    # 
    return dict(ngrams)

In [3]:
def test_count_ngrams():
    assert count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True) == {
        ('<s>',): 1, 
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=True) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=2, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1,
        ('привет', 'привет'): 1,
        ('привет', 'как'): 1,
        ('как', 'дела'): 1
    }    
    assert count_ngrams(['привет ' * 6], order=2, bos=False, eos=False) == {
        ('привет',): 6, 
        ('привет', 'привет'): 5
    }
    result = count_ngrams(['практическое сентября',
                           'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=5)
    assert result[('<s>',)] == 3
    assert result[('32',)] == 3
    assert result[('<s>', 'в', 'офлайне', 'в', '32')] == 1
    assert result[('офлайне', 'в', '32', '12', '</s>')] == 1
    print('Test 1a passed')
    
    
test_count_ngrams()  

Test 1a passed



Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 

Чтобы избежать данного недостатка, вводится специальное сглаживание - [сглаживание Лапласа](https://en.wikipedia.org/wiki/Additive_smoothing). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.


In [4]:
# функция подсчета вероятности через количество со сглаживанием Лапласа
def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
    # подсчитывет ngram со сглаживанием Лапласа
    # TODO
    vocab_size = 0
    head_count = 0
    for key, value in counts.items():
        if V is None and len(key) == 1:
            vocab_size+=1
        if len(key)==len(ngram):
            if key[:-1] == ngram[:-1]:
                head_count+=value
    V = vocab_size if V is None else V
    ngram_count = counts[ngram] if ngram in counts else 0
    prob = (ngram_count+k)/(head_count+k*V)
    return prob

In [5]:
def test_calculate_ngram_prob():
    counts = count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)
    assert calculate_ngram_prob(('в', 'офлайне'), counts) == 0.5
    assert calculate_ngram_prob(('в', ), counts) == 4/25
    assert calculate_ngram_prob(('в', ), counts, k=0.5) == (4+0.5)/(25+0.5*12)
    assert calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts) == 1.0
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=1) == 0.1875
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=0) == 0.0
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=1) == 0.0625
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

    print("Test 1.b passed")
    

test_calculate_ngram_prob()  

Test 1.b passed


Основной метрикой язковых моделей является перплексия. 

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$


In [6]:
# Языковая модель 
class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = eos
        self.bos = bos
        self.k = k
        self.vocab = predefined_vocab
        self.ngrams_count = None
        
    @property
    def V(self) -> int:
        return len(self.vocab)
    
    def _count_ngrams(self, train_text: List[str]) -> Dict[Tuple[str], int]:
        # подсчитывает все 1-gram 2-gram ... order-gram ngram'ы в тексте
        ngrams = defaultdict(int)
        for text in train_text:
            tokens = text.split()
            if self.bos:
                tokens.insert(0, "<s>")
            if self.eos:
                tokens.append("</s>")

            for i in range(self.order):
                n_gram = i + 1
                for j in range(len(tokens)-n_gram+1):
                    ngrams[tuple(tokens[j:j+n_gram])]+=1
        return dict(ngrams)
    
    def _calculate_ngram_prob(self, ngram: Tuple[str]) -> float:
        # подсчитывет ngram со сглаживанием Лапласа
        head_count = 0
        for key, value in self.ngrams_count.items():
            if len(key)==len(ngram):
                if key[:-1] == ngram[:-1]:
                    head_count+=value
        ngram_count = self.ngrams_count[ngram] if ngram in self.ngrams_count else 0
        prob = (ngram_count+self.k)/(head_count+self.k*self.V)
        return prob
    
    def fit(self, train_text: List[str]) -> None:
        # TODO
        # Подсчет vocab и ngrams_count по обучающей выборке
        self.ngrams_count = self._count_ngrams(train_text)
        self.vocab = [ ngram[0] for ngram in self.ngrams_count.keys() if len(ngram) == 1]                
    
    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        # TODO 
        # считаем логарифм вероятности конкретной нграмы
        return np.log(self._calculate_ngram_prob(ngram))
           
    def predict_log_proba(self, words: List[str]) -> float:
        if self.bos:
            words = ['<s>'] + words
        if self.eos:
            words = words + ['</s>']
        logprob = 0
        # TODO 
        # применяем chain rule, чтобы посчитать логарифм вероятности всей строки

        for i in range(len(words)):
            end = i + 1
            start = end - self.order
            actual_start = start if start > 0 else 0
            ngram = tuple(words[actual_start:end])
            logprob += self.predict_ngram_log_proba(ngram)
        
        return logprob
        
    def ppl(self, text: List[str]) -> float:
        #TODO 
        # подсчет перплексии
        # Для того, чтобы ваш код был численно стабильным, 
        #    не считайте формулу напрямую, а воспользуйтесь переходом к логарифмам вероятностей
        # 
        perplexity = 0
        for line in text:
            words = line.split()
            logprob = self.predict_log_proba(words)
            perplexity += logprob
        perplexity = np.exp(-perplexity / (len(words)+self.bos+self.eos))
        return perplexity

In [7]:
def test_lm():
    train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
    global lm
    lm = NgramLM(order=2)
    lm.fit(train_data)
    assert lm.V == 14
    assert np.isclose(lm.predict_log_proba(['мы']), lm.predict_log_proba(["если"]))
    assert lm.predict_log_proba(["по-моему"]) > lm.predict_log_proba(["если"]) 
    
    gt = ((3+1)/(41 + 14) * 1/(3+14))**(-1/2)
    ppl = lm.ppl([''])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    gt = ((3+1)/(41 + 14) * 1/(3+14) * 1/(14)) ** (-1/3)
    ppl = lm.ppl(['ЧТО'])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]
    ppl = lm.ppl(test_data)
    assert round(ppl, 2) == 7.33, f"{ppl}"
test_lm()

# 2. Предсказания с помощью языковой модели (4 балла)

In [8]:
def predict_next_word(lm: NgramLM, prefix: List[str], topk=4):
    # TODO реализуйте функцию, которая предсказывает продолжение фразы. 
    # верните topk наиболее вероятных продолжений фразы prefix 
    
    list_prob = []
    
    for word in lm.vocab:
        ngram = tuple(prefix+[word])
        log_prob = lm.predict_ngram_log_proba(ngram)
        list_prob.append((ngram, log_prob))
    
    list_prob.sort(key=lambda e: e[1], reverse=True)
    top_prob = list_prob[:topk]
    next_words = [(e[0][-1], e[1]) for e in top_prob ]
    return next_words

train_data = [
    "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
    "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
    "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"
]

lm = NgramLM(order=3)
lm.fit(train_data)

predict_next_word(lm, ["я"])

[('сойду', -1.7346010553881062),
 ('сейчас', -2.1400661634962708),
 ('<s>', -2.833213344056216),
 ('по-моему', -2.833213344056216)]

Обучите языковую модель на всем тексте из этой лабораторной работы (не забудьте заранее нормализовать текст).

Что предскажет модель по префиксам `по-моему`, `сэкономим`, `нграм` и `вероятности`? 

In [9]:
lab_train_text = """
Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. 

Практическая работа разделена на 2 части: 
1. Построение нграмой языковой модели - основная часть, 12 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 4 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

1. Построение нграмной языковой модели. (12 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: 
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$


Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 

Чтобы избежать данного недостатка, вводится специальное сглаживание - [сглаживание Лапласа](https://en.wikipedia.org/wiki/Additive_smoothing). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.

Основной метрикой язковых моделей является перплексия. 

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$

2. Предсказания с помощью языковой модели (4 балла)

Обучите языковую модель на всем тексте из этой лабораторной работы (не забудьте заранее нормализовать текст).

Что предскажет модель по префиксам `по-моему`, `сэкономим`, `нграм` и `вероятности`? 

"""

In [10]:
import re
def preprocess_sentence(sentence):
    sentence = sentence.lower().replace("ё", "е")
    sentence = re.sub('[^а-яА-Я-]+', ' ', sentence) 
    sentence = re.sub(' +', ' ',  sentence)
    return sentence.strip()

In [11]:
src_sentences = re.split(r"[\.!?]+|\n+", lab_train_text)
norm_sentences = [preprocess_sentence(sentence) for sentence in src_sentences]
norm_sentences = [sentence for sentence in norm_sentences if sentence != ""]

In [12]:
lm = NgramLM(order=3)
lm.fit(norm_sentences)

In [13]:
predict_next_word(lm, ["по-моему"])

[('мы', -4.238925791033208),
 ('сэкономим', -4.6443908991413725),
 ('<s>', -5.337538079701318),
 ('языковые', -5.337538079701318)]

In [14]:
predict_next_word(lm, ["сэкономим"])

[('уйму', -4.243721963296701),
 ('нграм', -4.6491870714048655),
 ('сэкономим', -4.6491870714048655),
 ('<s>', -5.342334251964811)]

In [15]:
predict_next_word(lm, ["нграм"])

[('и', -4.2626798770413155),
 ('</s>', -4.66814498514948),
 ('в', -4.66814498514948),
 ('языковой', -4.66814498514948)]

In [16]:
predict_next_word(lm, ["вероятности"])

[('</s>', -4.6443908991413725),
 ('для', -4.6443908991413725),
 ('хорошо', -4.6443908991413725),
 ('<s>', -5.337538079701318)]